# Entraîner un modèle de classification avec le Machine Learning automatisé

Il existe de nombreux types d’algorithmes d’apprentissage automatique que vous pouvez utiliser pour effectuer l’apprentissage d’un modèle, et il n’est pas toujours facile de déterminer l’algorithme le plus efficace pour vos données et vos exigences de prédiction spécifiques. En outre, vous pouvez affecter de manière significative les performances prédictives d’un modèle en prétraitant les données d’apprentissage à l’aide de techniques telles que la normalisation, l’imputation de fonctionnalités manquantes et autres. Dans votre quête du meilleur modèle pour vos besoins, vous devrez peut-être essayer de nombreuses combinaisons d’algorithmes et de transformations de prétraitement, ce qui monopolise beaucoup de temps et de ressources de calcul.

Azure Machine Learning vous permet d’automatiser la comparaison de modèles entraînés à l’aide de différents algorithmes et options de prétraitement. Vous pouvez utiliser l’interface visuelle dans [Azure Machine Learning Studio](https://ml/azure.com) ou le SDK Python (v2) pour tirer parti de cette fonctionnalité. Le SDK Python vous offre un plus grand contrôle sur les paramètres du travail de Machine Learning automatisé, mais l’interface visuelle est plus facile à utiliser.

## Avant de commencer

Vous devez avoir la dernière version du package **azureml-ai-ml** pour exécuter le code dans ce notebook. Exécutez la cellule ci-dessous pour vérifier qu’il est installé.

> **Remarque** :
> Si le package **azure-ai-ml** n’est pas installé, exécutez `pip install azure-ai-ml` pour l’installer.

In [ ]:
## Se connecter à un espace de travail

Une fois les packages requis du Kit de développement logiciel (SDK) installés, vous êtes prêt à vous connecter à votre espace de travail.

Pour vous connecter à un espace de travail, vous avez besoin de paramètres d’identificateur : un ID d’abonnement, un nom de groupe de ressources et un nom d’espace de travail. Le nom du groupe de ressources et le nom de l’espace de travail sont déjà renseignés pour vous. Vous avez seulement besoin de l’ID d’abonnement pour exécuter la commande.

Pour trouver les paramètres nécessaires, cliquez sur l’abonnement et le nom de l’espace de travail en haut à droite du studio. Un volet s’ouvre à droite.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiez l’ID d’abonnement et remplacez **YOUR-SUBSCRIPTION-ID** par la valeur que vous avez copiée. </p>

## Préparer les données

Vous n’avez pas besoin de créer un script d’entraînement pour le Machine Learning automatisé, mais vous devez charger les données d’entraînement. 

Dans le cas présent, vous allez utiliser un jeu de données contenant les détails de patients atteints de diabète. 

Pour passer un jeu de données en tant qu’entrée à un travail de Machine Learning automatisé, les données doivent être sous forme tabulaire et inclure une colonne cible. Pour que les données soient interprétées comme un jeu de données tabulaire, le jeu de données d’entrée doit être un **MLTable**.

Une ressource de données MLTable a déjà été créée pour vous pendant la configuration. Vous pouvez explorer la ressource de données en accédant à la page **Données**. Vous allez récupérer la ressource de données ici en spécifiant son nom `diabetes-training-table` et sa version `1`. 

In [ ]:
## Configurer le travail de Machine Learning automatisé

Vous êtes maintenant prêt à configurer l’expérience du Machine Learning automatisé.

Lorsque vous exécutez le code ci-dessous, celui-ci crée un travail de Machine Learning automatisé qui :

- Utilise le cluster de calcul nommé `aml-cluster`
- Définit `Diabetic` comme colonne cible
- Définit `accuracy` comme métrique principale
- Expire après `60` minutes de temps total d’entraînement 
- Entraîne un maximum de `5` modèles
- Aucun modèle ne sera entraîné avec l’algorithme `LogisticRegression`

In [ ]:
## Exécuter un travail de machine learning automatisé

OK, vous êtes prêt. Nous allons exécuter l’expérience de Machine Learning automatisé.

> **Remarque** : Cela peut prendre un certain temps !

Vous pouvez surveiller le travail dans le studio pendant qu’il s’exécute.

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named `aml-cluster`
- Sets `Diabetic` as the target column
- Sets `accuracy` as the primary metric
- Times out after `60` minutes of total training time 
- Trains a maximum of `5` models
- No model will be trained with the `LogisticRegression` algorithm

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

While the job is running, you can monitor it in the Studio.